In [11]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
import scikitplot as skplt
%matplotlib inline

In [12]:
def process_data():
    columns_to_named = ['Adj']
    ind = pd.read_excel(r"C:\Users\123\Downloads\mysql\SPY.csv",parse_dates=True)
    ind_norm = (ind['Adj']-ind['Adj'].mean())/ind['Adj'].std(ddof=0)
    x_data = ind_norm.to_frame()
    y = ind['Adj Close']
    kind=[]
    b=y.median()
    for i in y:

        if i>=b:
            kind.append(1)
        if i<b:
            kind.append(0)

    y = pd.DataFrame({'P':kind})
    y_data = y['P']
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.3, random_state = 101)
    return x_train, x_test, y_train, y_test

# Define Feature Map

In [13]:
def create_feature_column():
    feat_Adj = tf.feature_column.numeric_column('Adj')
    feature_column = [feat_Adj]
    return feature_column

In [14]:
x_train, x_test, y_train, y_test = process_data()
feature_column = create_feature_column()


In [15]:
input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(x_train, y_train, batch_size=50, num_epochs= 1000, shuffle=True)
#.numpy_input_fn(x_train, y_train, batch_size=50, num_epochs= 1000, shuffle=True)

In [16]:
eval_func = tf.compat.v1.estimator.inputs.pandas_input_fn(x_test, y_test, batch_size=50, num_epochs= 1, shuffle=False)
#.numpy_input_fn(x_test, y_test, batch_size=50, num_epochs= 1, shuffle=False)


In [17]:
predict_input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(x=x_test, num_epochs=1, shuffle=False)
#.inputs.numpy_input_fn(x=x_test, num_epochs=1, shuffle=False)

# Create a Model

In [18]:
dnnmodel = tf.estimator.DNNClassifier(hidden_units=[20,20],
                                      feature_columns=feature_column,
                                      n_classes=2,
                                      activation_fn=tf.nn.softmax,
                                      dropout= None,
                                      optimizer=tf.optimizers.Adam(learning_rate=0.01)
                                     )

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\123\\AppData\\Local\\Temp\\tmpy2k_0bi5', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# Train, Test, Evaluate

In [19]:
history = dnnmodel.train(input_fn=input_func,steps=500
                       )

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\123\AppData\Local\Temp\tmpy2k_0bi5\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.69423395, step = 0
INFO:tensorflow:global_step/sec: 536.028
INFO:tensorflow:loss = 0.531413, step = 100 (0.188 sec)
INFO:tensorflow:global_step/sec: 759.985
INFO:tensorflow:loss = 0.5456314, step = 200 (0.132 sec)
INFO:tensorflow:global_step/sec: 752.888
INFO:tensorflow:loss = 0.46685147, step = 300 (0.134 sec)
INFO:tensorflow:global_step/sec: 716.185
INFO:tensorflow:loss = 0.4462181, step = 400 (0.144 sec)
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 500...

In [20]:
dnnmodel.evaluate(eval_func)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-04-19T09:58:06Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\123\AppData\Local\Temp\tmpy2k_0bi5\model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.47322s
INFO:tensorflow:Finished evaluation at 2021-04-19-09:58:07
INFO:tensorflow:Saving dict for global step 500: accuracy = 0.73717946, accuracy_baseline = 0.52350426, auc = 0.8376224, auc_precision_recall = 0.839038, average_loss = 0.48366868, global_step = 500, label/mean = 0.4764957, loss = 0.48648733, precision = 0.710084, prediction/mean = 0.5067572, recall = 0.75784755
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 500: C:\Users\123\AppData\Local\Temp\tmpy2k_0bi5\model.ckpt-500


{'accuracy': 0.73717946,
 'accuracy_baseline': 0.52350426,
 'auc': 0.8376224,
 'auc_precision_recall': 0.839038,
 'average_loss': 0.48366868,
 'label/mean': 0.4764957,
 'loss': 0.48648733,
 'precision': 0.710084,
 'prediction/mean': 0.5067572,
 'recall': 0.75784755,
 'global_step': 500}